# Decorator based approach

In [111]:
import logging
from abc import ABC, abstractmethod
import pickle  # For deep copying complex states

# Configure logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

# ====== Memento Class ======
class Memento:
    def __init__(self, state: str):
        self._state = state

    def get_saved_state(self) -> str:
        return self._state

    def __repr__(self):
        return f"Memento(state='{self._state}')"

# ====== TextEditor Class (Originator/Receiver) ======
class TextEditor:
    def __init__(self):
        self._content = ""

    def write(self, text: str):
        self._content += text
        logging.info(f"Written text: '{text}'. Current content: '{self._content}'")

    def delete(self, count: int) -> str:
        start_index = max(0, len(self._content) - count)
        deleted = self._content[start_index:]
        self._content = self._content[:start_index]
        logging.info(f"Deleted last {count} characters: '{deleted}'. Current content: '{self._content}'")
        return deleted  # Return the deleted text for undo

    def save_state(self) -> Memento:
        logging.debug(f"Saving state to memento. State: '{self._content}'")
        return Memento(self._content)

    def restore_state(self, memento: Memento):
        self._content = memento.get_saved_state()
        logging.info(f"State restored from memento. Current content: '{self._content}'")

    def get_content(self) -> str:
        return self._content

    def __repr__(self):
        return f"TextEditor(content='{self._content}')"

# ====== Command Interface ======
class Command(ABC):
    @abstractmethod
    def execute(self):
        pass
    
class UndoableCommand(Command):
    @abstractmethod
    def undo(self):
        pass
    
# ====== Concrete Commands ======
class WriteCommand(UndoableCommand):
    def __init__(self, editor: TextEditor, text: str):
        self._editor = editor
        self._text = text
        self._length = len(text)  # Store the length of text added

    def execute(self):
        logging.debug(f"Executing WriteCommand with text: '{self._text}'")
        self._editor.write(self._text)

    def undo(self):
        logging.debug(f"Undoing WriteCommand. Deleting last {self._length} characters.")
        self._editor.delete(self._length)

    def __repr__(self):
        return f"WriteCommand(text='{self._text}', editor={repr(self._editor)})"

class DeleteCommand(UndoableCommand):
    def __init__(self, editor: TextEditor, count: int):
        self._editor = editor
        self._count = count
        self._deleted_text = ""  # Store the deleted text

    def execute(self):
        logging.debug(f"Executing DeleteCommand. Deleting {self._count} characters.")
        self._deleted_text = self._editor.delete(self._count)

    def undo(self):
        logging.debug(f"Undoing DeleteCommand. Restoring deleted text: '{self._deleted_text}'")
        self._editor.write(self._deleted_text)

    def __repr__(self):
        return f"DeleteCommand(count={self._count}, editor={repr(self._editor)})"


# ====== Invoker Class ======
class CommandManager:
    def __init__(self):
        self._undo_stack = []
        self._redo_stack = []
        
    def clear_stacks(self):
        self._undo_stack.clear()
        self._redo_stack.clear()
        logging.info("Undo and redo stacks cleared.")
        
    def execute_command(self, command: Command):
        logging.debug(f"Executing command: {type(command).__name__}")
        command.execute()
        
        # Only store undoable commands in the undo stack
        if hasattr(command, "undo"):
            self._undo_stack.append(command)
            self._redo_stack.clear()
            logging.info(f"Undoable command stored: {type(command).__name__}")
        else:
            logging.info(f"Command executed but not stored (not undoable): {type(command).__name__}")
        
        self._log_stack_states()

    def undo(self):
        if not self._undo_stack:
            logging.warning("No commands to undo.")
            return
        command = self._undo_stack.pop()
        command.undo()
        self._redo_stack.append(command)
        logging.info(f"Command undone: {type(command).__name__}")
        self._log_stack_states()

    def redo(self):
        if not self._redo_stack:
            logging.warning("No commands to redo.")
            return
        command = self._redo_stack.pop()
        command.execute()
        self._undo_stack.append(command)
        logging.info(f"Command redone: {type(command).__name__}")
        self._log_stack_states()

    def _log_stack_states(self):
        undo_states = [repr(cmd) for cmd in self._undo_stack]
        redo_states = [repr(cmd) for cmd in self._redo_stack]
        logging.debug(f"Undo stack: {undo_states}")
        logging.debug(f"Redo stack: {redo_states}")

    def __repr__(self):
        return f"CommandManager(undo_stack={self._undo_stack}, redo_stack={self._redo_stack})"

class Event:
    def __init__(self):
        self._subscribers = []

    def subscribe(self, callback):
        self._subscribers.append(callback)

    def emit(self, *args, **kwargs):
        for callback in self._subscribers:
            callback(*args, **kwargs)

# ====== Caretaker Class for Mementos ======
class DocumentManager:
    def __init__(self):
        self._saved_states = []
        self.on_state_restored = Event()  # Event for state restoration

    def save(self, memento: Memento):
        self._saved_states.append(memento)
        logging.info(f"Document saved. Total saved states: {len(self._saved_states)}")

    def load(self, index: int) -> Memento:
        if 0 <= index < len(self._saved_states):
            logging.info(f"Loading document from saved state {index}")
            memento = self._saved_states[index]
            self.on_state_restored.emit()  # Notify all subscribers
            return memento
        else:
            logging.error("Invalid index for saved states.")
            return None
        
    def save_editor_state(self, editor: TextEditor):
        memento = editor.save_state()
        self.save(memento)
        
    def restore_editor_state(self, editor: TextEditor, index: int):
        memento = self.load(index)  # Fetch the memento
        if memento:
            editor.restore_state(memento)
            
    def __repr__(self):
        return f"DocumentManager(saved_states={self._saved_states})"
    
class SaveCommand(Command):
    def __init__(self, editor: TextEditor, document_manager: DocumentManager):
        self.editor = editor
        self.document_manager = document_manager

    def execute(self):
        memento = self.editor.save_state()
        self.document_manager.save(memento)
        
        
class RestoreCommand(Command):
    def __init__(self, editor: TextEditor, document_manager: DocumentManager, index: int):
        self.editor = editor
        self.document_manager = document_manager
        self.index = index

    def execute(self):
        memento = self.document_manager.load(self.index)
        if memento:
            self.editor.restore_state(memento)
        
        


In [112]:
# A hypothetical logger class to demonstrate another feature
class DocumentLogger:
    def log_restore(self):
        print("Document state has been restored. Logging this action.")
        
# ====== Decorator Classes ======
class CommandManagerDecorator:
    def __init__(self, command_manager: CommandManager):
        self._command_manager = command_manager

    def execute_command(self, command: Command):
        self._command_manager.execute_command(command)

    def undo(self):
        self._command_manager.undo()

    def redo(self):
        self._command_manager.redo()

    def clear_stacks(self):
        self._command_manager.clear_stacks()

    def __repr__(self):
        return f"{self.__class__.__name__}({repr(self._command_manager)})"

class AutoSaveDecorator(CommandManagerDecorator):
    def __init__(self, command_manager: CommandManager, editor: TextEditor, document_manager: DocumentManager, threshold: int):
        super().__init__(command_manager)
        self._editor = editor
        self._document_manager = document_manager
        self._threshold = threshold
        self._command_count = 0

    def execute_command(self, command: Command):
        super().execute_command(command)
        self._command_count += 1
        if self._command_count >= self._threshold:
            self._command_count = 0
            # Perform autosave
            memento = self._editor.save_state()
            self._document_manager.save(memento)
            logging.info("Autosave performed after reaching the command threshold.")

    def __repr__(self):
        return f"AutoSaveDecorator(threshold={self._threshold}, editor={repr(self._editor)}, document_manager={repr(self._document_manager)}, command_manager={repr(self._command_manager)})"


In [113]:
def main():
    editor = TextEditor()
    command_manager = CommandManager()
    document_manager = DocumentManager()
    logger = DocumentLogger()
    command_manager = AutoSaveDecorator(
    command_manager=command_manager,
    editor=editor,
    document_manager=document_manager,
    threshold=2  # Autosave after every 3 commands
)
    # Subscribe CommandManager to the DocumentManager's event
    document_manager.on_state_restored.subscribe(command_manager.clear_stacks)
    
    # Subscribe the logger's log_restore method to the same event
    document_manager.on_state_restored.subscribe(logger.log_restore)
    
    # Define a sequence of automated commands
    commands = [
        ("write", "Hello, "),
        ("show", None),  
        ("write", "World!"),
        ("show", None),  
        ("write", " My b."),
        ("show", None),  
        ("write", " Actually nah.."),
        ("show", None), 
        ("delete", 6),
        ("show", None),
        ("undo", None),
        ("show", None),
        ("load", 0),
        ("show", None),
        ("load", 1),
        ("show", None)
    ]


    # Execute predefined commands
    for action, value in commands:
        print(f"Executing: {action}{f' {value}' if value is not None else ''}")
        if action == "write":
            cmd = WriteCommand(editor, value)
            command_manager.execute_command(cmd)
        elif action == "delete":
            cmd = DeleteCommand(editor, value)
            command_manager.execute_command(cmd)
        elif action == "undo":
            command_manager.undo()
        elif action == "redo":
            command_manager.redo()
        elif action == "save":
            save_cmd = SaveCommand(editor, document_manager)
            command_manager.execute_command(save_cmd)  # Save via command
        elif action == "load":
            restore_cmd = RestoreCommand(editor, document_manager, value)
            command_manager.execute_command(restore_cmd)  # Restore via command
        elif action == "show":
            print(f"Current content: '{editor.get_content()}'")
        print("-" * 50)


if __name__ == "__main__":
    main()

2024-11-26 15:41:03,434 - DEBUG - Executing command: WriteCommand
2024-11-26 15:41:03,435 - DEBUG - Executing WriteCommand with text: 'Hello, '
2024-11-26 15:41:03,437 - INFO - Written text: 'Hello, '. Current content: 'Hello, '
2024-11-26 15:41:03,439 - INFO - Undoable command stored: WriteCommand
2024-11-26 15:41:03,441 - DEBUG - Undo stack: ["WriteCommand(text='Hello, ', editor=TextEditor(content='Hello, '))"]
2024-11-26 15:41:03,443 - DEBUG - Redo stack: []
2024-11-26 15:41:03,445 - DEBUG - Executing command: WriteCommand
2024-11-26 15:41:03,448 - DEBUG - Executing WriteCommand with text: 'World!'
2024-11-26 15:41:03,449 - INFO - Written text: 'World!'. Current content: 'Hello, World!'
2024-11-26 15:41:03,451 - INFO - Undoable command stored: WriteCommand
2024-11-26 15:41:03,453 - DEBUG - Undo stack: ["WriteCommand(text='Hello, ', editor=TextEditor(content='Hello, World!'))", "WriteCommand(text='World!', editor=TextEditor(content='Hello, World!'))"]
2024-11-26 15:41:03,454 - DEBUG 

Executing: write Hello, 
--------------------------------------------------
Executing: show
Current content: 'Hello, '
--------------------------------------------------
Executing: write World!
--------------------------------------------------
Executing: show
Current content: 'Hello, World!'
--------------------------------------------------
Executing: write  My b.
--------------------------------------------------
Executing: show
Current content: 'Hello, World! My b.'
--------------------------------------------------
Executing: write  Actually nah..
--------------------------------------------------
Executing: show
Current content: 'Hello, World! My b. Actually nah..'
--------------------------------------------------
Executing: delete 6
--------------------------------------------------
Executing: show
Current content: 'Hello, World! My b. Actually'
--------------------------------------------------
Executing: undo
--------------------------------------------------
Executing: show


# Event based approach

In [ ]:
import logging
from abc import ABC, abstractmethod
import pickle  # For deep copying complex states

# Configure logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

# ====== Memento Class ======
class Memento:
    def __init__(self, state: str):
        self._state = state

    def get_saved_state(self) -> str:
        return self._state

    def __repr__(self):
        return f"Memento(state='{self._state}')"

# ====== TextEditor Class (Originator/Receiver) ======
class TextEditor:
    def __init__(self):
        self._content = ""

    def write(self, text: str):
        self._content += text
        logging.info(f"Written text: '{text}'. Current content: '{self._content}'")

    def delete(self, count: int) -> str:
        start_index = max(0, len(self._content) - count)
        deleted = self._content[start_index:]
        self._content = self._content[:start_index]
        logging.info(f"Deleted last {count} characters: '{deleted}'. Current content: '{self._content}'")
        return deleted  # Return the deleted text for undo

    def save_state(self) -> Memento:
        logging.debug(f"Saving state to memento. State: '{self._content}'")
        return Memento(self._content)

    def restore_state(self, memento: Memento):
        self._content = memento.get_saved_state()
        logging.info(f"State restored from memento. Current content: '{self._content}'")

    def get_content(self) -> str:
        return self._content

    def __repr__(self):
        return f"TextEditor(content='{self._content}')"

# ====== Command Interface ======
class Command(ABC):
    @abstractmethod
    def execute(self):
        pass
    
class UndoableCommand(Command):
    @abstractmethod
    def undo(self):
        pass
    
# ====== Concrete Commands ======
class WriteCommand(UndoableCommand):
    def __init__(self, editor: TextEditor, text: str):
        self._editor = editor
        self._text = text
        self._length = len(text)  # Store the length of text added

    def execute(self):
        logging.debug(f"Executing WriteCommand with text: '{self._text}'")
        self._editor.write(self._text)

    def undo(self):
        logging.debug(f"Undoing WriteCommand. Deleting last {self._length} characters.")
        self._editor.delete(self._length)

    def __repr__(self):
        return f"WriteCommand(text='{self._text}', editor={repr(self._editor)})"

class DeleteCommand(UndoableCommand):
    def __init__(self, editor: TextEditor, count: int):
        self._editor = editor
        self._count = count
        self._deleted_text = ""  # Store the deleted text

    def execute(self):
        logging.debug(f"Executing DeleteCommand. Deleting {self._count} characters.")
        self._deleted_text = self._editor.delete(self._count)

    def undo(self):
        logging.debug(f"Undoing DeleteCommand. Restoring deleted text: '{self._deleted_text}'")
        self._editor.write(self._deleted_text)

    def __repr__(self):
        return f"DeleteCommand(count={self._count}, editor={repr(self._editor)})"


# ====== Invoker Class ======
class CommandManager:
    def __init__(self):
        self._undo_stack = []
        self._redo_stack = []
        self.on_command_executed = Event()  # Event for command execution
        
    def add_command_executed_listener(self, callback):
        """Add a listener for the command executed event."""
        self.on_command_executed.subscribe(callback)
        
    def clear_stacks(self):
        self._undo_stack.clear()
        self._redo_stack.clear()
        logging.info("Undo and redo stacks cleared.")
        
    def execute_command(self, command: Command):
        logging.debug(f"Executing command: {type(command).__name__}")
        command.execute()
        
        # Only store undoable commands in the undo stack
        if hasattr(command, "undo"):
            self._undo_stack.append(command)
            self._redo_stack.clear()
            logging.info(f"Undoable command stored: {type(command).__name__}")
        else:
            logging.info(f"Command executed but not stored (not undoable): {type(command).__name__}")
        
        self._log_stack_states()
        # Emit the command executed event
        self.on_command_executed.emit(command=command)
        
        
    def undo(self):
        if not self._undo_stack:
            logging.warning("No commands to undo.")
            return
        command = self._undo_stack.pop()
        command.undo()
        self._redo_stack.append(command)
        logging.info(f"Command undone: {type(command).__name__}")
        self._log_stack_states()

    def redo(self):
        if not self._redo_stack:
            logging.warning("No commands to redo.")
            return
        command = self._redo_stack.pop()
        command.execute()
        self._undo_stack.append(command)
        logging.info(f"Command redone: {type(command).__name__}")
        self._log_stack_states()

    def _log_stack_states(self):
        undo_states = [repr(cmd) for cmd in self._undo_stack]
        redo_states = [repr(cmd) for cmd in self._redo_stack]
        logging.debug(f"Undo stack: {undo_states}")
        logging.debug(f"Redo stack: {redo_states}")

    def __repr__(self):
        return f"CommandManager(undo_stack={self._undo_stack}, redo_stack={self._redo_stack})"

class Event:
    def __init__(self):
        self._subscribers = []

    def subscribe(self, callback):
        self._subscribers.append(callback)

    def emit(self, *args, **kwargs):
        for callback in self._subscribers:
            callback(*args, **kwargs)

# ====== Caretaker Class for Mementos ======
class DocumentManager:
    def __init__(self):
        self._saved_states = []
        self.on_state_restored = Event()  # Event for state restoration
        
    def add_state_restored_listener(self, callback):
        """Subscribe a callback to the state restored event."""
        self.on_state_restored.subscribe(callback)
        
    def save(self, memento: Memento):
        self._saved_states.append(memento)
        logging.info(f"Document saved. Total saved states: {len(self._saved_states)}")

    def load(self, index: int) -> Memento:
        if 0 <= index < len(self._saved_states):
            logging.info(f"Loading document from saved state {index}")
            memento = self._saved_states[index]
            self.on_state_restored.emit()  # Notify all subscribers
            return memento
        else:
            logging.error("Invalid index for saved states.")
            return None
        
    def save_editor_state(self, editor: TextEditor):
        memento = editor.save_state()
        self.save(memento)
        
    def restore_editor_state(self, editor: TextEditor, index: int):
        memento = self.load(index)  # Fetch the memento
        if memento:
            editor.restore_state(memento)
            
    def __repr__(self):
        return f"DocumentManager(saved_states={self._saved_states})"
    
class SaveCommand(Command):
    def __init__(self, editor: TextEditor, document_manager: DocumentManager):
        self.editor = editor
        self.document_manager = document_manager

    def execute(self):
        memento = self.editor.save_state()
        self.document_manager.save(memento)
        
        
class RestoreCommand(Command):
    def __init__(self, editor: TextEditor, document_manager: DocumentManager, index: int):
        self.editor = editor
        self.document_manager = document_manager
        self.index = index

    def execute(self):
        memento = self.document_manager.load(self.index)
        if memento:
            self.editor.restore_state(memento)
        
        

In [115]:
# A hypothetical logger class to demonstrate another feature
class DocumentLogger:
    def log_restore(self):
        print("Document state has been restored. Logging this action.")
        
class AutosaveHandler:
    def __init__(self, editor: TextEditor, document_manager: DocumentManager, threshold: int):
        self._editor = editor
        self._document_manager = document_manager
        self._threshold = threshold
        self._command_count = 0

    def on_command_executed(self, command: Command):
        if not hasattr(command, "undo"):
            return
        # Increment the command count
        self._command_count += 1
        logging.info(f"Command executed. Current command count: {self._command_count}")

        # Check if the threshold is met
        if self._command_count >= self._threshold:
            self._command_count = 0  # Reset the counter
            self._perform_autosave()

    def _perform_autosave(self):
        # Perform the autosave operation
        memento = self._editor.save_state()
        self._document_manager.save(memento)
        logging.info("Autosave performed.")

    def __repr__(self):
        return f"AutosaveHandler(threshold={self._threshold}, editor={repr(self._editor)}, document_manager={repr(self._document_manager)})"


In [116]:
def main():
    editor = TextEditor()
    command_manager = CommandManager()
    document_manager = DocumentManager()
    logger = DocumentLogger()
    # Create the AutosaveHandler with a threshold of 3 commands
    autosave_handler = AutosaveHandler(editor, document_manager, threshold=2)

    # Subscribe the AutosaveHandler to the CommandManager's event
    command_manager.add_command_executed_listener(autosave_handler.on_command_executed)
    
    # Subscribe CommandManager to the DocumentManager's event
    document_manager.add_state_restored_listener(command_manager.clear_stacks)
    
    # Subscribe the logger's log_restore method to the same event
    document_manager.add_state_restored_listener(logger.log_restore)
    
    # Define a sequence of automated commands
    commands = [
        ("write", "Hello, "),
        ("show", None),  
        ("write", "World!"),
        ("show", None),  
        ("write", " My b."),
        ("show", None),  
        ("write", " Actually nah.."),
        ("show", None), 
        ("delete", 6),
        ("show", None),
        ("undo", None),
        ("show", None),
        ("load", 0),
        ("show", None),
        ("load", 1),
        ("show", None)
    ]


    # Execute predefined commands
    for action, value in commands:
        print(f"Executing: {action}{f' {value}' if value is not None else ''}")
        if action == "write":
            cmd = WriteCommand(editor, value)
            command_manager.execute_command(cmd)
        elif action == "delete":
            cmd = DeleteCommand(editor, value)
            command_manager.execute_command(cmd)
        elif action == "undo":
            command_manager.undo()
        elif action == "redo":
            command_manager.redo()
        elif action == "save":
            save_cmd = SaveCommand(editor, document_manager)
            command_manager.execute_command(save_cmd)  # Save via command
        elif action == "load":
            restore_cmd = RestoreCommand(editor, document_manager, value)
            command_manager.execute_command(restore_cmd)  # Restore via command
        elif action == "show":
            print(f"Current content: '{editor.get_content()}'")
        print("-" * 50)


if __name__ == "__main__":
    main()



2024-11-26 15:41:03,856 - DEBUG - Executing command: WriteCommand
2024-11-26 15:41:03,857 - DEBUG - Executing WriteCommand with text: 'Hello, '
2024-11-26 15:41:03,859 - INFO - Written text: 'Hello, '. Current content: 'Hello, '
2024-11-26 15:41:03,860 - INFO - Undoable command stored: WriteCommand
2024-11-26 15:41:03,861 - DEBUG - Undo stack: ["WriteCommand(text='Hello, ', editor=TextEditor(content='Hello, '))"]
2024-11-26 15:41:03,862 - DEBUG - Redo stack: []
2024-11-26 15:41:03,865 - INFO - Command executed. Current command count: 1
2024-11-26 15:41:03,867 - DEBUG - Executing command: WriteCommand
2024-11-26 15:41:03,869 - DEBUG - Executing WriteCommand with text: 'World!'
2024-11-26 15:41:03,871 - INFO - Written text: 'World!'. Current content: 'Hello, World!'
2024-11-26 15:41:03,872 - INFO - Undoable command stored: WriteCommand
2024-11-26 15:41:03,874 - DEBUG - Undo stack: ["WriteCommand(text='Hello, ', editor=TextEditor(content='Hello, World!'))", "WriteCommand(text='World!', ed

Executing: write Hello, 
--------------------------------------------------
Executing: show
Current content: 'Hello, '
--------------------------------------------------
Executing: write World!
--------------------------------------------------
Executing: show
Current content: 'Hello, World!'
--------------------------------------------------
Executing: write  My b.
--------------------------------------------------
Executing: show
Current content: 'Hello, World! My b.'
--------------------------------------------------
Executing: write  Actually nah..
--------------------------------------------------
Executing: show
Current content: 'Hello, World! My b. Actually nah..'
--------------------------------------------------
Executing: delete 6
--------------------------------------------------
Executing: show
Current content: 'Hello, World! My b. Actually'
--------------------------------------------------
Executing: undo
--------------------------------------------------
Executing: show
